In [10]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [11]:
y = train['label']
X = train.drop('label', axis = 1)
X /= 255

train_idx, val_idx = 29400, 29400 + 6300

X_train, y_train =  X.iloc[:train_idx].values.reshape(-1,28, 28, 1),            y.iloc[:train_idx].values
X_val, y_val =      X.iloc[train_idx:val_idx].values.reshape(-1, 28, 28, 1),    y.iloc[train_idx:val_idx].values
X_test, y_test =    X.iloc[val_idx:].values.reshape(-1, 28, 28, 1),             y.iloc[val_idx:].values

print(X_train.shape)

(29400, 28, 28, 1)


In [12]:
import tensorflow as tf
import tensorflow.keras.layers as tfl
from sklearn.metrics import accuracy_score
np.random.seed(1)

In [13]:
model = tf.keras.Sequential([
    tfl.Flatten(input_shape = (28, 28, 1)), 
    tfl.Dense(128, activation= 'relu'), 
    tfl.Dense(64, activation = 'relu'), 
    tfl.Dense(32, activation= 'relu'), 
    tfl.Dense(10, activation = 'softmax')
])

model.compile(optimizer = 'adam', 
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits= False), 
              metrics = ['accuracy'])

model.fit(X_train, y_train, epochs = 50, verbose= 1)

val_pred_probs = model.predict(X_val)
val_pred = tf.argmax(val_pred_probs, axis=1).numpy()
val_accuracy = accuracy_score(y_val, val_pred)
print('Accuracy on the validation set:', round(val_accuracy, 4))

Epoch 1/50


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


919/919 ━━━━━━━━━━━━━━━━━━━━ 1s 929us/step - accuracy: 0.8060 - loss: 0.6198
Epoch 2/50
919/919 ━━━━━━━━━━━━━━━━━━━━ 1s 917us/step - accuracy: 0.9540 - loss: 0.1553
Epoch 3/50
919/919 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9704 - loss: 0.0936
Epoch 4/50
919/919 ━━━━━━━━━━━━━━━━━━━━ 1s 820us/step - accuracy: 0.9757 - loss: 0.0759
Epoch 5/50
919/919 ━━━━━━━━━━━━━━━━━━━━ 1s 900us/step - accuracy: 0.9841 - loss: 0.0541
Epoch 6/50
919/919 ━━━━━━━━━━━━━━━━━━━━ 1s 883us/step - accuracy: 0.9868 - loss: 0.0417
Epoch 7/50
919/919 ━━━━━━━━━━━━━━━━━━━━ 1s 809us/step - accuracy: 0.9899 - loss: 0.0334
Epoch 8/50
919/919 ━━━━━━━━━━━━━━━━━━━━ 1s 790us/step - accuracy: 0.9913 - loss: 0.0266
Epoch 9/50
919/919 ━━━━━━━━━━━━━━━━━━━━ 1s 767us/step - accuracy: 0.9911 - loss: 0.0277
Epoch 10/50
919/919 ━━━━━━━━━━━━━━━━━━━━ 1s 731us/step - accuracy: 0.9926 - loss: 0.0224
Epoch 11/50
919/919 ━━━━━━━━━━━━━━━━━━━━ 1s 747us/step - accuracy: 0.9938 - loss: 0.0203
Epoch 12/50
919/919 ━━━━━━━━━━━━━━━━━━━━ 1s

In [14]:
test_values = test.values.reshape(-1, 28, 28, 1)
test_pred_probs = model.predict(test_values)
test_pred = tf.argmax(test_pred_probs, axis = 1).numpy()
test_df = pd.DataFrame(test_pred, index = test.index, columns = ['Label'])
test_df.index.name = 'ImageId'
test_df.index = test_df.index + 1
#test_df.to_csv('simple_nn_v1.csv')

875/875 ━━━━━━━━━━━━━━━━━━━━ 0s 269us/step


In [15]:
conv = tf.keras.Sequential()

conv.add(tfl.Conv2D(filters= 28, kernel_size= (5, 5), padding = 'same', input_shape = X_train.shape[1:], activation = 'relu'))
conv.add(tfl.BatchNormalization())

conv.add(tfl.Conv2D(filters = 28, kernel_size= (5,5), activation = 'relu'))
conv.add(tfl.MaxPool2D(pool_size = (3,3)))

conv.add(tfl.Conv2D(filters = 32, kernel_size = (5,5), padding = 'same', activation = 'relu'))
conv.add(tfl.BatchNormalization())
conv.add(tfl.Conv2D(filters = 32, kernel_size = (5,5), activation = 'relu'))
conv.add(tfl.MaxPool2D(pool_size = (2,2)))

conv.add(tfl.Flatten())
conv.add(tfl.Dense(512, activation = 'relu'))
conv.add(tfl.Dense(64, activation = 'relu'))
conv.add(tfl.Dense(24, activation = 'relu'))
conv.add(tfl.Dense(10, activation = 'softmax'))

conv.compile(optimizer= 'adam',
             loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits= False), 
             metrics = ['accuracy'])

conv.fit(X_train, y_train, epochs = 20, verbose = 1)


val_pred_probs = conv.predict(X_val)
val_pred = tf.argmax(val_pred_probs, axis=1).numpy()
val_accuracy = accuracy_score(y_val, val_pred)
print('Accuracy on the validation set:', round(val_accuracy, 4))

Epoch 1/20


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


919/919 ━━━━━━━━━━━━━━━━━━━━ 17s 17ms/step - accuracy: 0.8683 - loss: 0.3981
Epoch 2/20
919/919 ━━━━━━━━━━━━━━━━━━━━ 16s 17ms/step - accuracy: 0.9830 - loss: 0.0608
Epoch 3/20
919/919 ━━━━━━━━━━━━━━━━━━━━ 16s 18ms/step - accuracy: 0.9865 - loss: 0.0477
Epoch 4/20
919/919 ━━━━━━━━━━━━━━━━━━━━ 16s 17ms/step - accuracy: 0.9898 - loss: 0.0371
Epoch 5/20
919/919 ━━━━━━━━━━━━━━━━━━━━ 16s 17ms/step - accuracy: 0.9914 - loss: 0.0294
Epoch 6/20
919/919 ━━━━━━━━━━━━━━━━━━━━ 16s 17ms/step - accuracy: 0.9915 - loss: 0.0262
Epoch 7/20
919/919 ━━━━━━━━━━━━━━━━━━━━ 16s 17ms/step - accuracy: 0.9942 - loss: 0.0200
Epoch 8/20
919/919 ━━━━━━━━━━━━━━━━━━━━ 17s 18ms/step - accuracy: 0.9931 - loss: 0.0227
Epoch 9/20
919/919 ━━━━━━━━━━━━━━━━━━━━ 16s 18ms/step - accuracy: 0.9943 - loss: 0.0197
Epoch 10/20
919/919 ━━━━━━━━━━━━━━━━━━━━ 17s 18ms/step - accuracy: 0.9959 - loss: 0.0153
Epoch 11/20
919/919 ━━━━━━━━━━━━━━━━━━━━ 17s 18ms/step - accuracy: 0.9951 - loss: 0.0179
Epoch 12/20
919/919 ━━━━━━━━━━━━━━━━━━━━ 

In [19]:
test_pred_probs = conv.predict(test_values)
test_pred = tf.argmax(test_pred_probs, axis = 1).numpy()
test_df = pd.DataFrame(test_pred, index = test.index, columns = ['Label'])
test_df.index.name = 'ImageId'
test_df.index = test_df.index + 1
test_df.to_csv('convolutional_nn_v1.csv')

875/875 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step
